<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!mkdir spacy_ner_data

In [3]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/100_resumes_annotated.json', "r") as f:
    data = json.load(f)

def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # Only add if there's no overlap with the previous entity
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        entities = remove_overlapping_entities(entities)  # Remove overlapping entities
        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [ ]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [ ]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# train model using hyperparameters set in config.cfg
# save trained model in spacy-output/ dir
!python -m spacy train config.cfg --output ./spacy-output

ℹ Saving to output directory: spacy-output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    311.54    0.00    0.00    0.00    0.00
  2     200        727.25  13365.40   30.91   46.99   23.03    0.31
  5     400       1523.59   9725.39   39.50   41.25   37.88    0.39
  7     600        209.56   6266.99   40.28   36.05   45.65    0.40
 10     800        342.38   4632.03   41.02   37.42   45.38    0.41
 12    1000       1274.97   3608.79   38.70   42.97   35.21    0.39
 15    1200        224.03   2632.41   41.37   42.51   40.29    0.41
 17    1400        246.44   2273.57   40.72   42.02   39.49    0.41
 20    1600        240.64

In [ ]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate spacy-output/model-best spacy_ner_data/test_data.spacy -dp spacy-output

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   42.51 
NER R   40.29 
NER F   41.37 
SPEED   2090  


=============================== NER (per type) ===============================

                 P        R       F
NAME        100.00    66.67   80.00
PHONE        92.86    72.22   81.25
EMAIL        69.23   100.00   81.82
SKILL        31.60    30.10   30.83
JOB          44.62    52.73   48.33
WORK PER     72.13    77.19   74.58
COMPANY      44.74    41.46   43.04
DEG          82.35    73.68   77.78
STUDY PER   100.00    37.50   54.55
UNI          43.75    41.18   42.42
LOC         100.00    33.33   50.00

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-p

In [ ]:
# make prediction
import spacy
resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''
nlp = spacy.load("spacy-output/model-best")
doc = nlp(resume_text.lower())

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

(john doe lives at, john.doe@example.com, software engineer, august 2019, at tech, july 2017 to july 2019, aws, – associate, a, cloud architect)
john doe lives at: COMPANY
john.doe@example.com: EMAIL
software engineer: JOB
august 2019: WORK PER
at tech: COMPANY
july 2017 to july 2019: WORK PER
aws: SKILL
– associate: JOB
a: SKILL
cloud architect: JOB


In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# download trained model

## Flair NER

In [4]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=fd8f5f883327cccaa0908bf2335d4ce6fc93c8cc780b0f0f07cdaf8b8b36483d
  Stored in directory: /root/.cach

In [13]:
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

In [ ]:
import spacy
from spacy.training import Corpus

!python -m spacy download de_core_news_sm
nlp = spacy.load("de_core_news_sm")
corpus = Corpus("/content/spacy_ner_data/test_data.spacy")

data = corpus(nlp)

# Flair supports BIO and BIOES, see https://github.com/flairNLP/flair/issues/875
def rename_biluo_to_bioes(old_tag):
    new_tag = ""
    try:
        if old_tag.startswith("L"):
            new_tag = "E" + old_tag[1:]
        elif old_tag.startswith("U"):
            new_tag = "S" + old_tag[1:]
        else:
            new_tag = old_tag
    except:
        pass
    return new_tag


def generate_corpus():
    corpus = []
    n_ex = 0
    for example in data:
        n_ex += 1
        text = example.text
        doc = nlp(text)
        tags = example.get_aligned_ner()
        # Check if it's an empty list of NER tags.
        if None in tags:
            pass
        else:
            new_tags = [rename_biluo_to_bioes(tag) for tag in tags]
            for token, tag in zip(doc,new_tags):
                row = token.text +' '+ token.pos_ +' ' +tag + '\n'
                corpus.append(row)
            corpus.append('\n')
    return corpus

def write_file(filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        corpus = generate_corpus()
        f.writelines(corpus)

def main():
    write_file('flair_test.txt')

main()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file)

2024-11-03 01:04:41,893 Reading data from .
2024-11-03 01:04:41,895 Train: flair_train.txt
2024-11-03 01:04:41,897 Dev: None
2024-11-03 01:04:41,899 Test: flair_test.txt
2024-11-03 01:04:42,458 No dev split found. Using 10% (i.e. 8 samples) of the train split as dev data


In [15]:
tag_dictionary= corpus.make_label_dictionary(label_type = 'ner')
tag_dictionary

2024-11-03 01:04:45,218 Computing label dictionary. Progress:


0it [00:00, ?it/s]
72it [00:00, 8012.89it/s]

2024-11-03 01:04:45,248 Dictionary created for label 'ner' with 11 values: SKILL (seen 1242 times), JOB (seen 230 times), WORK (seen 225 times), COMPANY (seen 186 times), UNI (seen 76 times), NAME (seen 70 times), DEG (seen 67 times), STUDY (seen 66 times), PHONE (seen 63 times), EMAIL (seen 46 times), LOC (seen 43 times)


In [16]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger

embeddings = StackedEmbeddings([
                WordEmbeddings('glove'),
                WordEmbeddings('en-crawl')
            ])

tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=tag_dictionary,
                         tag_type='ner',
                         use_crf=True)

2024-11-03 01:05:29,639 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-UNI, B-UNI, E-UNI, I-UNI, S-NAME, B-NAME, E-NAME, I-NAME, S-DEG, B-DEG, E-DEG, I-DEG, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, S-LOC, B-LOC, E-LOC, I-LOC


In [ ]:
# train flair ner model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train(
    base_path='flair_output/',
    learning_rate=0.01,
    mini_batch_size=8,
    max_epochs=10,
    patience=3,
    embeddings_storage_mode='gpu',
    use_amp=True,  # Use mixed precision training
    train_with_dev=False
)
!cp -r ./flair_output /content/drive/MyDrive/FYP/Implementation/

2024-11-03 01:06:06,734 ----------------------------------------------------------------------------------------------------
2024-11-03 01:06:06,737 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=400, out_features=400, bias=True)
  (rnn): LSTM(400, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=47, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-11-03 01:06:06,739 ----------------------------------------------------------------------------------------------------
2024-11-03 01:06:06,741 Corpus: 72 train + 8 dev + 20 test sentences
2024-11-03 01:06:06,743 -----------------------------------

100%|██████████| 1/1 [00:04<00:00,  4.86s/it]

2024-11-03 01:24:22,367 DEV : loss 2.8770179748535156 - f1-score (micro avg)  0.038
2024-11-03 01:24:22,372  - 0 epochs without improvement
2024-11-03 01:24:22,375 saving best model


2024-11-03 01:24:50,118 ----------------------------------------------------------------------------------------------------
2024-11-03 01:28:22,823 epoch 2 - iter 1/9 - loss 2.81432092 - time (sec): 212.70 - samples/sec: 20.21 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:30:06,159 epoch 2 - iter 2/9 - loss 2.80048909 - time (sec): 316.04 - samples/sec: 26.21 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:31:57,828 epoch 2 - iter 3/9 - loss 2.77881974 - time (sec): 427.71 - samples/sec: 28.78 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:33:17,795 epoch 2 - iter 4/9 - loss 2.74596366 - time (sec): 507.67 - samples/sec: 30.89 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:35:53,734 epoch 2 - iter 5/9 - loss 2.69784780 - time (sec): 663.61 - samples/sec: 29.79 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:39:44,770 epoch 2 - iter 6/9 - loss 2.65911166 - time (sec): 894.65 - samples/sec: 27.09 - lr: 0.010000 - momentum: 0.000000
2024-11-03 01:41:39,221 epoch 2 - iter 7/

100%|██████████| 1/1 [00:05<00:00,  5.38s/it]

...

2024-11-03 03:23:42,001 DEV : loss 0.7899544835090637 - f1-score (micro avg)  0.0
2024-11-03 03:23:42,008  - 1 epochs without improvement
2024-11-03 03:23:42,010 ----------------------------------------------------------------------------------------------------


2024-11-03 03:26:32,840 epoch 8 - iter 1/9 - loss 0.61237529 - time (sec): 170.82 - samples/sec: 27.05 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:29:54,904 epoch 8 - iter 2/9 - loss 0.69264520 - time (sec): 372.89 - samples/sec: 22.68 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:31:41,208 epoch 8 - iter 3/9 - loss 0.71922312 - time (sec): 479.19 - samples/sec: 25.07 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:33:41,719 epoch 8 - iter 4/9 - loss 0.74521172 - time (sec): 599.70 - samples/sec: 25.62 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:36:15,979 epoch 8 - iter 5/9 - loss 0.74590742 - time (sec): 753.96 - samples/sec: 25.50 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:37:10,900 epoch 8 - iter 6/9 - loss 0.78587192 - time (sec): 808.88 - samples/sec: 27.93 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:39:04,106 epoch 8 - iter 7/9 - loss 0.80332199 - time (sec): 922.09 - samples/sec: 28.69 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:42:38,214 epo

100%|██████████| 1/1 [00:05<00:00,  5.05s/it]

2024-11-03 03:44:21,816 DEV : loss 0.7730231285095215 - f1-score (micro avg)  0.0
2024-11-03 03:44:21,822  - 2 epochs without improvement
2024-11-03 03:44:21,825 ----------------------------------------------------------------------------------------------------


2024-11-03 03:46:44,408 epoch 9 - iter 1/9 - loss 0.85003544 - time (sec): 142.58 - samples/sec: 22.84 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:50:47,767 epoch 9 - iter 2/9 - loss 0.74345527 - time (sec): 385.94 - samples/sec: 22.78 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:51:39,418 epoch 9 - iter 3/9 - loss 0.79316960 - time (sec): 437.59 - samples/sec: 25.25 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:53:41,708 epoch 9 - iter 4/9 - loss 0.80357288 - time (sec): 559.88 - samples/sec: 27.13 - lr: 0.005000 - momentum: 0.000000
2024-11-03 03:55:31,636 epoch 9 - iter 5/9 - loss 0.80587715 - time (sec): 669.81 - samples/sec: 27.87 - lr: 0.005000 - momentum: 0.000000


In [ ]:
# evaluate model
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Load the trained model
model = SequenceTagger.load('flair_output/best-model.pt')

# Evaluate the model on the test set
result = model.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)

# Print the results
# print("Evaluation Loss:", eval_loss)
print(result.detailed_results)  # This will print the precision, recall, and F1-score per entity type

2024-10-31 10:05:42,406 SequenceTagger predicts: Dictionary with 4 tags: O, PER, <START>, <STOP>


100%|██████████| 1/1 [00:03<00:00,  3.62s/it]


Results:
- F-score (micro) 0.6732
- F-score (macro) 0.6732
- Accuracy 0.5074

By class:
              precision    recall  f1-score   support

         PER     0.8519    0.5565    0.6732       124

   micro avg     0.8519    0.5565    0.6732       124
   macro avg     0.8519    0.5565    0.6732       124
weighted avg     0.8519    0.5565    0.6732       124



In [ ]:
# make prediction
import flair
model = SequenceTagger.load('flair_output/best-model.pt')
resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''
sentence = flair.data.Sentence(resume_text.lower())

model.predict(sentence)

print(sentence.to_tagged_string())

2024-10-31 10:06:35,880 SequenceTagger predicts: Dictionary with 4 tags: O, PER, <START>, <STOP>
Sentence[326]: " john doe lives at 1234 elm street in los angeles, ca 90001. he can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. john is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of javascript, python, and cloud technologies like aws and azure. currently, he works as a software engineer at google llc in san francisco, ca, where he has been employed since august 2019. in this role, he has developed scalable web applications using javascript, node.js, and react, deployed and maintained cloud infrastructure on aws, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. previously, he worked as a junior developer at tech innovators inc. in austin, tx, from july 2017 to july 2019, where he created restful apis using python and flask, collabo